In [12]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])
scaler = MinMaxScaler()
base_scaled = scaler.fit_transform(base)
base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = scaler.fit_transform(target[:5])
target_unscaled = scaler.inverse_transform(target_scaled)

num_clusters = 3
#print(target_scaled)

/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [20]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

[[ 0.47068966]
 [ 0.10098522]
 [ 0.90229885]]
0.117515995697


In [21]:
def cal_centroids(alpha,beta):

    target_centroids = []
    for i in range(len(base_centroids)):
        val = alpha*base_centroids[i][0] + beta
        target_centroids.append([val])

    target_centroids = np.array(target_centroids)
    return target_centroids
    #print(target_centroids)

In [22]:
def cal_gradient():

SyntaxError: unexpected EOF while parsing (<ipython-input-22-96e2af2db05b>, line 1)

In [23]:
sum_m = 0
sum_d = 0
for i in range(len(base_centroids)):
    sum_m = sum_m + base_centroids[i][0]
    
avg_m = sum_m/len(base_centroids)

for i in range(len(target_scaled)):
    sum_d = sum_d + target_scaled[i][0]

print(sum_m,sum_d)
avg_d = sum_d/len(target_scaled)

alpha = avg_d/avg_m
beta = 0
print(alpha)
        


1.47397372742 1.42
0.578029298724


In [24]:
kmeans = KMeans(n_clusters=num_clusters, init=cal_centroids(alpha,beta), max_iter=1)
kmeans.fit(target_scaled)
old_error = kmeans.inertia_

# while(1):
    
#     alpha = alpha - 0.01*
#     beta = beta + 1
#     kmeans = KMeans(n_clusters=num_clusters, init=cal_parameters(alpha,beta), max_iter=1)
#     kmeans.fit(target_unscaled)
#     new_error = kmeans.inertia_
#     print(old_error,new_error)
#     if(new_error < old_error):
#         old_error = new_error
                
#     else:
#         break;
    
    

#     #base_centroids = kmeans.cluster_centers_
#     #labels = kmeans.labels_

#     #print(kmeans.cluster_centers_)
#     print(old_error,new_error)

/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [25]:
from collections import Counter

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

c = Counter(labels)

clusters = []
for i in range(0,3):
    clusters.append([])
    
#print(kmeans.cluster_centers_)
for i in range(len(target_scaled)):
    #print("coordinate:",X[i], "label:", labels[i], "keyword:", keywords[i])
    clusters[labels[i]].append(target_scaled[i])

#unscaled_centroids = scaler.inverse_transform(centroids)
for i in range(0,3):
    print(centroids[i])
    print(clusters[i])
    print("******")

[ 0.3]
[array([ 0.3])]
******
[ 0.04]
[array([ 0.]), array([ 0.1]), array([ 0.02])]
******
[ 1.]
[array([ 1.])]
******
